In [10]:
import numpy as np #For array managment
import math
from IPython import display
import time
import random

from rover_grid import Grid

In [11]:
def DKL(l1,l2):
    ###Kullback-Leibler divergence between two arrays, they have to be pmfs (ie histograms that sum to 1)
    return(np.sum([l1[i]*math.log(l1[i]/l2[i]) for i in range(len(l1)) if l1[i]!=0]))

In [12]:
S = 5 

In [13]:
id_square = -1
actions = []

def find_min(weights,grid):
    
    min = np.min(weights)
    global id_square,actions

    # CHECK IF THE ROVER IS IN A NEW SQUARE
    if id_square != grid.get_square().get_id():
        actions = []    #RESET THE ACTIONS' VECTOR

    # CHECK IF THE ACTIONS' VECTOR IS EMPTY AND 
    # INITIALIZE IT WITH THE ACTIONS THAT HAVE THE LOWER WEIGHT
    if len(actions) == 0:
        i=1
        id_square = grid.get_square().get_id()
        for w in weights[1:]:
            if(w == min):
                actions.append(i)
            i+=1
        actions.append(grid.get_last_action()) # save the last action that moved the rover in a new state

    # IF THERE IS MORE THEN ONE ACTION WITH THE LOWER WEIGHT, 
    # CHOSE A RANDOM ACTION (EXCLUDING THE LAST 
    # ACTION MADE BY ROVER TO MOVE TO A NEW SQUARE)
    if len(actions) != 1:
        action = random.choice(actions[:-1])
        actions.remove(action)
        return action
    else: 
        # IF THE ROVER MADE ALL THE POSSIBLE ACTIONS AND DON'T 
        # CHANGE SQUARE THE ROVER RETURN ON THE LAST SQUARE.
        action = actions[0]
        actions.remove(action)
        return action


## TASK: Write a routine to validate your code.

In [14]:
timeHorizon = 5

def routine(rover_grid,receding_horizon_algorithm = False):
    path = []
    pmf_path = []

    global timeHorizon

    path.append("coordinates: "+str(rover_grid.get_current_position())+" square: "+str(rover_grid.get_current_square_id()))

    display.clear_output(wait=True)
    rover_grid.print_grid()
    time.sleep(0.2)

    while(rover_grid.is_on_target() == False):

        display.clear_output(wait=True)

        if(receding_horizon_algorithm):
            pmf = receding_horizon_crowdsourcing(timeHorizon,rover)
        else:
            pmf = greedy_find_step(rover_grid)
            
        action = np.argmax(pmf)
        rover_grid.move_rover(action)
        rover_grid.print_grid()
        path.append("Coordinates: "+str(rover_grid.get_current_position())+" Square: "+ str(rover_grid.get_current_square_id()))
        pmf_path.append(pmf)
        time.sleep(0.2)

    display.clear_output(wait=True)
    rover_grid.print_grid()

    print("\n----------------- PATH TO TARGET -----------------")

    print("The rover_grid starts from " + path[0])
    print("PMF: "+ str(pmf_path[0]))
    print("\n")

    for i in range(1,len(path)):
        print(path[i])
        if(i<len(pmf_path)):
            print("PMF: "+ str(pmf_path[i]))
        print("\n")


## TASK WP2 GREEDY ALGORITHM

In [ ]:
def greedy_find_step(grid):
    
    weights = np.zeros(S)

    pos_r, pos_c = grid.get_current_position()

    # CALCULATE THE REWARDS LINKED TO THE CURRENT POSITION OF THE 
    # ROVER, SO AS TO UNDERSTAND WHICH CAN BE THE BEST ACTION TO DO
    reward = grid.return_reward(pos_r, pos_c)

    # REPRESENTS ROVER'S BEHAVIOR IN THE SQUARE X FOR EACH ACTION, TAKING INTO ACCOUNT THE PRESENCE OF WALL
    sources = grid.return_sources(pos_r,pos_c)

    # WITH TARGET INSTEAD WE GET THE PMF WITH THE BEST ACTION TO REACH THE TARGET (EXLUDING THE PRESENCE OF OBSTACLES)
    target = grid.return_target(pos_r, pos_c)

    # COMPUTE THE VECTOR OF WEIGHTS FOR EACH POSSIBLE ACTION IN THE CURRENT SQUARE
    for i in range(S):
        a = DKL(sources[i],target) - np.dot(sources[i],reward)
        weights[i] = a
    
    # MAKE THE GREEDY CHOICE
    j = find_min(weights,grid)

    # RETURN THE PMF OF THE ACTION CHOSEN IN THE GREEDY CHOICE, TAKING INTO ACCOUNT THE PRESENCE OF THE OBSTACLES
    pmf = grid.return_sources(pos_r, pos_c)[j]
    return pmf

rover = Grid(3,3,2,0)
rover.generate_obstacle(1,1)

routine(rover)

## TASK WP4: RECEDING HORIZON

In [ ]:
xDim = 5

def receding_horizon_crowdsourcing(tHor, grid):
    #Main loop, calculate the weights through a receding horizon and choose a policy
    #The idea is that we restric the state space to the relative states available to us and use crowdsourcing on this new state space
    timeHor = list(range(tHor))
    timeHor.reverse()
    rHat = [0]*xDim #Reward modifier, same dimension as the relative state space
    weights = np.zeros((xDim, S)) #We have S weights per relative state

    if tHor == 0: #If the time horizon is zero (greedy case), we have to explicitly tell python to still do a loop
        timeHor = [0]
        
    for k in timeHor:
        for x in grid.available_States(): #In the restricted state space
            pos_r, pos_c = grid.actualState(x)
            rHat[x] = -min(weights[x]) #Calulate \hat{r}

            r = np.add(grid.return_reward(pos_r,pos_c),rHat) #Modify the reward array with \hat{r} by an elementwise multiplication
            
            sources = grid.return_sources(pos_r,pos_c) #Get the sources and the target
            target = grid.return_target(pos_r,pos_c)
            for i in range(S): #Main loop
                a = DKL(sources[i],target) - np.dot(sources[i],r) #Calculate the weight and store it
                weights[x][i] = a
        
        if k==0: #If we're at the last iteration
            j = find_min(weights[0],grid) #Get the minimum weight
            posRov = grid.get_current_position()
            return(grid.return_sources(posRov[0],posRov[1])[j]) #Return the corresponding policy

rover = Grid(10,10,9,0)
rover.generate_obstacle(9,7)
rover.generate_obstacle(7,9)
rover.generate_obstacle(7,2)
rover.generate_obstacle(1,7)
rover.generate_obstacle(9,3)
rover.generate_obstacle(4,5)
rover.generate_obstacle(5,4)
rover.generate_obstacle(5,5)
rover.generate_obstacle(2,3)
rover.generate_obstacle(0,7)
rover.generate_obstacle(2,4)
rover.generate_obstacle(3,1)
rover.generate_obstacle(2,1)
rover.generate_obstacle(4,4)
rover.generate_obstacle(0,3)
rover.generate_obstacle(1,4)
rover.generate_obstacle(2,5)
rover.generate_obstacle(7,1)
rover.generate_obstacle(5,8)
rover.generate_obstacle(2,7)

routine(rover,True)

## TASK WP5: LARGE GRID

In [ ]:
rover = Grid(10,10,9,0)
rover.generate_obstacle(9,7)
rover.generate_obstacle(7,9)
rover.generate_obstacle(7,2)
rover.generate_obstacle(1,7)
rover.generate_obstacle(9,3)
rover.generate_obstacle(4,5)
rover.generate_obstacle(5,4)
rover.generate_obstacle(5,5)
rover.generate_obstacle(2,3)
rover.generate_obstacle(0,7)
rover.generate_obstacle(2,4)
rover.generate_obstacle(3,1)
rover.generate_obstacle(2,1)
rover.generate_obstacle(4,4)
rover.generate_obstacle(0,3)
rover.generate_obstacle(1,4)
rover.generate_obstacle(2,5)
rover.generate_obstacle(7,1)
rover.generate_obstacle(5,8)
rover.generate_obstacle(2,7)

routine(rover)